In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D,Activation, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
import tensorflow as tf

In [2]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10

In [3]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [4]:
resnet50_model = Sequential()

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(32,32,3),
    pooling='max',
    classes=10,
)

for layer in pretrained_model.layers:
    layer.trainable=False

resnet50_model.add(pretrained_model)
resnet50_model.add(Flatten())
resnet50_model.add(Dense(3072, activation='relu'))
resnet50_model.add(Dense(2048, activation='relu'))
resnet50_model.add(Dense(10, activation='softmax'))

94765736/94765736 [==============================] - 43s 0us/step


In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)

In [6]:
from sklearn.model_selection import learning_curve


resnet50_model.compile(optimizer=SGD(0.01, momentum = 0.7), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
from keras.callbacks import ModelCheckpoint, CSVLogger
ckpt = ModelCheckpoint('saved_model/resnet50_model.hdf5')
csv = CSVLogger('saved_model/resnet50_log.csv')

In [10]:
history = resnet50_model.fit(datagen.flow(x_train, y_train, batch_size), validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size, callbacks = [csv, ckpt])
resnet_model50.save_weights('saved_model/10epochs_resnet50_cifar10.h5')

Epoch 1/10
391/391 [==============================] - 179s 458ms/step - loss: 1.3298 - accuracy: 0.5255 - val_loss: 1.1917 - val_accuracy: 0.5815
Epoch 2/10
391/391 [==============================] - 174s 444ms/step - loss: 1.2445 - accuracy: 0.5590 - val_loss: 1.1713 - val_accuracy: 0.5905
Epoch 3/10
391/391 [==============================] - 176s 450ms/step - loss: 1.1910 - accuracy: 0.5804 - val_loss: 1.1260 - val_accuracy: 0.6027
Epoch 4/10
391/391 [==============================] - 184s 470ms/step - loss: 1.1653 - accuracy: 0.5849 - val_loss: 1.1435 - val_accuracy: 0.5977
Epoch 5/10
391/391 [==============================] - 166s 423ms/step - loss: 1.1412 - accuracy: 0.5965 - val_loss: 1.0777 - val_accuracy: 0.6210
Epoch 6/10
391/391 [==============================] - 180s 461ms/step - loss: 1.1145 - accuracy: 0.6040 - val_loss: 1.0968 - val_accuracy: 0.6178
Epoch 7/10
391/391 [==============================] - 164s 420ms/step - loss: 1.0975 - accuracy: 0.6117 - val_loss: 1.0917 -

In [11]:
train_mse = resnet50_model.evaluate(x_train, y_train, verbose=0)
test_mse = resnet50_model.evaluate(x_test, y_test, verbose=0)

In [12]:
print('Train loss : %.3f %% , Test loss : %.3f %%' % (train_mse[0]*100, test_mse[0]*100))
print('Train accuracy : %.3f %% , Test accuracy : %.3f %%' % (train_mse[1]*100, test_mse[1]*100))

Train loss : 94.969 % , Test loss : 109.905 %
Train accuracy : 66.214 % , Test accuracy : 61.510 %
